In [1]:
import fitz
import string
import pandas as pd

In [2]:
filepath = r'REP-EDC-2020_Fusion_Final-1-5.pdf'

In [3]:
##
# Get the entire file as string
##
text_dict = []
with fitz.open(filepath) as doc:
    for page in doc:
        text_dict.append(page.get_text("dict", sort=False))

In [5]:
def removePuncandSpace(text):
    """
    Removes punctuation and spaces from a string
    """
    return text.translate(str.maketrans('', '', string.punctuation)).strip()

In [6]:
##
# Get an idea of looping through text
# For every page
# Extract additional information about the text as well: font and font size
# Store in list of dictionaries
##

org_list = []
org_id = -1
org_name = False

for count_page, page in enumerate(text_dict):
    for count_block_list, block_list in enumerate(page["blocks"]):
        for count_line_list, line_list in enumerate(block_list["lines"]):
            for count_spans_list, spans_list in enumerate(line_list["spans"]):
                                
                #Remove empty text
                if spans_list['text'].isspace():
                    continue
                
                #Check if font & size are that of org number or new org
                if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                    #Check if start of new org
                    try :
                        #Throws ValueError if name of org
                        int(spans_list['text'])
                    except ValueError:
                        #Only triggers when name of org
                        org_list[org_id]['Name'] = spans_list['text'].strip()                   
                    else:
                        #If not name of org then org number
                        if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                            org_list.append({'id' : spans_list['text'].strip()})   
                            org_id += 1
                
                #Check if font & size are that of org address
                #Uses round to filter more text: other text has size that rounds to 8
                if (spans_list['font'] == 'Helvetica') & (round(float(spans_list['size'])) == 9):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    #If key Address doesn't already exist, create it
                    if 'Address' not in org_list[org_id].keys():
                        org_list[org_id]['Address'] = ''
                        org_list[org_id]['Address'] += spans_list['text']
                        org_address = True
                    else:
                        #Strip here to avoid unnecessary blank space
                        #Maybe handle this later?
                        org_list[org_id]['Address'] += spans_list['text'].strip()
                
                #Check if font & size are that of field name
                if (spans_list['font'] == 'ArialNarrow') & (int(float(spans_list['size'])) == 8):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    #If key field doesn't already exist, create it. Checks if length string > 1 to remove bad text
                    if (removePuncandSpace(spans_list['text']) not in org_list[org_id].keys()) & (len(removePuncandSpace(spans_list['text'])) > 1):
                        org_list[org_id][removePuncandSpace(spans_list['text'])] = ''
                
                #Check if font & size are that of field text
                if (spans_list['font'] == 'Helvetica-Bold') & (round(float(spans_list['size'])) == 8):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    
                    #Place in last dict key: will always be something there, non-generalizable method
                    org_list[org_id][list( org_list[org_id])[-1]] += spans_list['text']
                    
               

                

In [9]:
extractedtext_5pages = pd.DataFrame(org_list)

In [10]:
extractedtext_5pages

,id,Name,Address,Secteur industriel,Langue de comm,DDD,FAF,N° de télCie,n° de tél,Site Web,...,N° de faxCie,2e n° de tél,Courriel Cie,Fax,2e contact pour,Filiale de,Domaines dintérêt,Projets privilégiés,Nbre de succ,Princip filiales
0,1,3M Canada inc.,"7290, rue Frederick Banting Saint-Laurent QC ...",Équipement et services industriels; Fabricatio...,Français,En tout temps,31 décembre,514-336-5252,800-265-1840,https://www.3mcanada.ca/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A & D Prévost,"305, 12e Avenue Richelieu QC J3L3T2","Matériaux de construction, manufacturiers; Fab...",Français,En tout temps,31 décembre,450-658-8771,NaN,www.adprevost.ca,...,450-658-0077,800-361-4433,info@prevost-architectural.com,450-658-0077,"DonsMadame Marie-Josée Dery, Coordonnatrice Ca...",NaN,NaN,NaN,NaN,NaN
2,3,AAR Aicraft - Services Trois-Rivières,"3750, chemin de l'aéroport Trois-Rivières QC...","Offre des services d'entretien, de réparation ...",Français,En tout temps,31 décembre,819-377-4500,NaN,https://www.aarcorp.com,...,819- 668-8811,NaN,comptabilite@aarcorp.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ABB (Albert Bob Bob),800 boul. Hymes Saint-Laurent QC H4S0B5,Équipement et services industriels; Fabriquer ...,Français,En tout temps,NaN,514-856-6222,NaN,http://new.abb.com/ca,...,514-856-6297,NaN,NaN,514-856-6297,NaN,Baldor-Dodge-Reliance,NaN,NaN,NaN,NaN
4,5,ABB Canada,800 boul. Hymes Saint-Laurent QC H4S0B5,Commerce de gros et détail; Grossistes-distrib...,Français,En tout temps,31 décembre,438-843-6000,888-856-6266,www.ABB.com/ca,...,514-856-6297,NaN,contact.center@ca.abb.com,514-856-6297,NaN,ABB Bomem,NaN,NaN,NaN,NaN
5,6,AbbVie,"8401, rte Transcanadienne Saint-Laurent Québe...",Produits pharmaceutiques et biotechnologiques;...,Anglais,NaN,NaN,514-906-9700,888-703-3006,http://www.abbvie.ca/fr/index.html,...,888-704-8266,NaN,NaN,NaN,NaN,NaN,,,NaN,NaN
6,7,"ABS,","7950, Vauban Montréal QC H1J2X5",Services en surveillance & contrôle de réalis...,Français,En tout temps,31 décembre,450-454-5644,866-454-5644,http://www.groupeabs.com/,...,514-493-6228,NaN,info@groupeabs.com,514-493-6228,NaN,NaN,NaN,NaN,NaN,NaN
7,8,ACCEO Solutions,"75, rue Queen, bur. 6100 Montréal QC H3C2N6",Informatique et logiciels; Spécialiste des éch...,Français,En tout temps,31 décembre,514-288-7161,NaN,http://www.acceo.com/,...,514-868-5334,800-471-7161,info@acceo.com,514-868-5334,NaN,NaN,NaN,NaN,,"Conceptum Inc., GFI Conseil Inc., GFI Solution..."
8,9,Accovia Inc.,"75, rue Queen, bur. 6100 Montréal QC H3C2N6",Informatique et logiciels; Expert dans le dépl...,Français,En tout temps,NaN,514-931-4433,NaN,http://acceo.com,...,514-931-4818,NaN,info@accovia.fr,514-931-4818,NaN,"Accovia France SARL, ACCEO Solutions",NaN,NaN,NaN,NaN
9,10,Acier AGF,"2270, rue Garneau Longueuil QC J4G1E7",Produits d'acier et de métal; Concepteur et fa...,Français,En tout temps,NaN,450-442-9494,1-877-404-9494,www.acieragf.com,...,450-442-0538,NaN,NaN,450-442-0538,NaN,NaN,NaN,NaN,,NaN


In [12]:
filepath2 = r'REP-EDC-2020_Fusion_Final.pdf'

In [13]:
##
# Get the entire file as string
##
text_dict_full = []
with fitz.open(filepath2) as doc:
    for page in doc:
        text_dict_full.append(page.get_text("dict", sort=False))

In [15]:
##
# Get an idea of looping through text
# For every page
# Extract additional information about the text as well: font and font size
# Store in list of dictionaries
##

org_list = []
org_id = -1
org_name = False

for count_page, page in enumerate(text_dict_full):
    for count_block_list, block_list in enumerate(page["blocks"]):
        for count_line_list, line_list in enumerate(block_list["lines"]):
            for count_spans_list, spans_list in enumerate(line_list["spans"]):
                                
                #Remove empty text
                if spans_list['text'].isspace():
                    continue
                
                #Check if font & size are that of org number or new org
                if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                    #Check if start of new org
                    try :
                        #Throws ValueError if name of org
                        int(spans_list['text'])
                    except ValueError:
                        #Only triggers when name of org
                        org_list[org_id]['Name'] = spans_list['text'].strip()                   
                    else:
                        #If not name of org then org number
                        if (spans_list['font'] == 'Helvetica-Bold') & (int(float(spans_list['size'])) == 11):
                            org_list.append({'id' : spans_list['text'].strip()})   
                            org_id += 1
                
                #Check if font & size are that of org address
                #Uses round to filter more text: other text has size that rounds to 8
                if (spans_list['font'] == 'Helvetica') & (round(float(spans_list['size'])) == 9):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    #If key Address doesn't already exist, create it
                    if 'Address' not in org_list[org_id].keys():
                        org_list[org_id]['Address'] = ''
                        org_list[org_id]['Address'] += spans_list['text']
                        org_address = True
                    else:
                        #Strip here to avoid unnecessary blank space
                        #Maybe handle this later?
                        org_list[org_id]['Address'] += spans_list['text'].strip()
                
                #Check if font & size are that of field name
                if (spans_list['font'] == 'ArialNarrow') & (int(float(spans_list['size'])) == 8):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    #If key field doesn't already exist, create it. Checks if length string > 1 to remove bad text
                    if (removePuncandSpace(spans_list['text']) not in org_list[org_id].keys()) & (len(removePuncandSpace(spans_list['text'])) > 1):
                        org_list[org_id][removePuncandSpace(spans_list['text'])] = ''
                
                #Check if font & size are that of field text
                if (spans_list['font'] == 'Helvetica-Bold') & (round(float(spans_list['size'])) == 8):
                    #Catch if no orgs created
                    if org_id < 0:
                        continue
                    
                    #Place in last dict key: will always be something there, non-generalizable method
                    org_list[org_id][list( org_list[org_id])[-1]] += spans_list['text']
                    
               

                

In [ ]:
extractedtext_full = pd.DataFrame(org_list)

In [ ]:
extractedtext_full